In [16]:
import pandas as pd
from rapidfuzz import process, fuzz
import re

# 加载数据
hospital_data_path = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-1\上交版本7-1陈昕宇.xlsx'
hospital_data = pd.read_excel(hospital_data_path, sheet_name='医院基础信息核对 （上交版本）')
insurance_data = pd.read_excel(hospital_data_path, sheet_name='医院基础信息核对 （上交版本）')

hospital_data['地址'] = hospital_data['地址'].fillna('')
insurance_data['地址'] = insurance_data['地址'].fillna('')

hospital_data = hospital_data[hospital_data['地址'].apply(len) > 6]
insurance_data = insurance_data[insurance_data['地址'].apply(len) > 6]

# 从医院名称中提取关键术语
def extract_key_terms(name):
    # 移除常见后缀并提取关键术语
    key_terms = re.sub(r'(医院|中心|门诊部|保健中心|服务中心|诊所)', '', name)
    key_terms = re.split(r'[市县区]', key_terms)
    key_terms = [term for term in key_terms if term]
    return key_terms

# 应用提取函数
hospital_data['KeyTerms'] = hospital_data['医院名称'].apply(extract_key_terms)
insurance_data['KeyTerms'] = insurance_data['医院名称'].apply(extract_key_terms)

# 根据名称相似性识别重复项的函数
def identify_name_duplicates(row, insurance_df):
    hospital_key_terms = row['KeyTerms']
    potential_matches = insurance_df[insurance_df['KeyTerms'].apply(lambda x: sum(term in x for term in hospital_key_terms) >= 2)]
    if not potential_matches.empty:
        # 排除当前行自身
        potential_matches = potential_matches[potential_matches['医院名称'] != row['医院名称']]
        # 比较地址以确认重复项
        if not potential_matches.empty:
            matches = process.extract(row['医院名称'], potential_matches['医院名称'], scorer=fuzz.WRatio, limit=5)
            result = []
            for match in matches:
                if match[1] > 50:  # 名称相似性的阈值
                    matched_row = potential_matches[potential_matches['医院名称'] == match[0]].iloc[0]
                    result.append((matched_row['医院名称'], match[1]))
            if result:
                result = result[:5]  # 保证结果数量为5
                result += [(None, 0)] * (5 - len(result))  # 填充不足的部分
                return result
    return [(None, 0)] * 5

# 应用名称重复项识别函数
name_matches = hospital_data.apply(lambda row: identify_name_duplicates(row, insurance_data), axis=1)
hospital_data['名称类似的医保名称1'] = name_matches.apply(lambda x: x[0][0])
hospital_data['名称匹配度1'] = name_matches.apply(lambda x: x[0][1])
hospital_data['名称类似的医保名称2'] = name_matches.apply(lambda x: x[1][0])
hospital_data['名称匹配度2'] = name_matches.apply(lambda x: x[1][1])
hospital_data['名称类似的医保名称3'] = name_matches.apply(lambda x: x[2][0])
hospital_data['名称匹配度3'] = name_matches.apply(lambda x: x[2][1])
hospital_data['名称类似的医保名称4'] = name_matches.apply(lambda x: x[3][0])
hospital_data['名称匹配度4'] = name_matches.apply(lambda x: x[3][1])
hospital_data['名称类似的医保名称5'] = name_matches.apply(lambda x: x[4][0])
hospital_data['名称匹配度5'] = name_matches.apply(lambda x: x[4][1])

# 根据地址相似性识别重复项的函数
def identify_address_duplicates(row, insurance_df, exclude_list):
    matches = process.extract(row['地址'], insurance_df[~insurance_df['医院名称'].isin(exclude_list)]['地址'], scorer=fuzz.WRatio, limit=5)
    result = []
    for match in matches:
        if match[1] > 80:  # 地址相似性的阈值
            matched_row = insurance_df[insurance_df['地址'] == match[0]].iloc[0]
            if matched_row['医院名称'] != row['医院名称']:
                result.append((matched_row['医院名称'], matched_row['地址'], match[1]))
    if result:
        result = result[:5]  # 保证结果数量为5
        result += [(None, None, 0)] * (5 - len(result))  # 填充不足的部分
        return result
    return [(None, None, 0)] * 5

# 应用地址重复项识别函数
address_matches = hospital_data.apply(lambda row: identify_address_duplicates(row, insurance_data, hospital_data['名称类似的医保名称1'].dropna().unique()), axis=1)
hospital_data['地址类似的医保名称1'] = address_matches.apply(lambda x: x[0][0])
hospital_data['地址类似的医保地址1'] = address_matches.apply(lambda x: x[0][1])
hospital_data['地址匹配度1'] = address_matches.apply(lambda x: x[0][2])
hospital_data['地址类似的医保名称2'] = address_matches.apply(lambda x: x[1][0])
hospital_data['地址类似的医保地址2'] = address_matches.apply(lambda x: x[1][1])
hospital_data['地址匹配度2'] = address_matches.apply(lambda x: x[1][2])
hospital_data['地址类似的医保名称3'] = address_matches.apply(lambda x: x[2][0])
hospital_data['地址类似的医保地址3'] = address_matches.apply(lambda x: x[2][1])
hospital_data['地址匹配度3'] = address_matches.apply(lambda x: x[2][2])
hospital_data['地址类似的医保名称4'] = address_matches.apply(lambda x: x[3][0])
hospital_data['地址类似的医保地址4'] = address_matches.apply(lambda x: x[3][1])
hospital_data['地址匹配度4'] = address_matches.apply(lambda x: x[3][2])
hospital_data['地址类似的医保名称5'] = address_matches.apply(lambda x: x[4][0])
hospital_data['地址类似的医保地址5'] = address_matches.apply(lambda x: x[4][1])
hospital_data['地址匹配度5'] = address_matches.apply(lambda x: x[4][2])

# 选择要显示或保存的列
name_selected_columns = [
    '医院名称', '名称类似的医保名称1', '名称匹配度1', 
    '名称类似的医保名称2', '名称匹配度2', 
    '名称类似的医保名称3', '名称匹配度3', 
    '名称类似的医保名称4', '名称匹配度4', 
    '名称类似的医保名称5', '名称匹配度5'
]

address_selected_columns = [
    '医院名称', '地址', '地址类似的医保地址1', '地址类似的医保名称1', '地址匹配度1',
    '地址类似的医保地址2', '地址类似的医保名称2', '地址匹配度2',
    '地址类似的医保地址3', '地址类似的医保名称3', '地址匹配度3',
    '地址类似的医保地址4', '地址类似的医保名称4', '地址匹配度4',
    '地址类似的医保地址5', '地址类似的医保名称5', '地址匹配度5'
]

df_name_out = hospital_data[name_selected_columns]
df_address_out = hospital_data[address_selected_columns]

# 可选的，将结果保存到Excel文件
output_path_name = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-2\name_output.xlsx'
df_name_out.to_excel(output_path_name, index=False)

output_path_address = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-2\address_output.xlsx'
df_address_out.to_excel(output_path_address, index=False)
